# Pre

In [1]:
from __future__ import annotations
# %pip uninstall mediocreatbest
%pip install --upgrade --force-reinstall \
    mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
try:
    from mediocreatbest import auto
except ImportError:
    %pip install --quiet --upgrade pip
    %pip install --upgrade --force-reinstall \
        mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
    from mediocreatbest import auto

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to /tmp/pip-install-9wh6nmt4/mediocreatbest_9b1828f8998c4dceb29ca7a12abc105d
  Running command git clone --filter=blob:none --quiet https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git /tmp/pip-install-9wh6nmt4/mediocreatbest_9b1828f8998c4dceb29ca7a12abc105d
  Resolved https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to commit e41d8d653f5889dd403fed60135cfcf0de585704
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mediocreatbest: filename=mediocreatbest-0.2.33-py3-none-any.whl size=16365 sha256=31e82a229a3aec77304df9242631753481b9d23d618baf79b98042eb95eca03a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hfkzc1ug/wheels/cc/6d/dd/d4137c3485df0cb8773c136ecc231252590bf217127892aaa5


# Config

In [5]:
config = auto.types.SimpleNamespace()

/auto.pprint.pp config width=144

namespace()


# Lib

## Address to lat/lng

In [11]:
from opencage.geocoder import OpenCageGeocode
def dms_to_decimal(dms_str):
    # Regular expression to parse the DMS format
    match = auto.re.match(r"(\d+)° (\d+)' ([\d.]+)'' ([NSEW])", dms_str)
    if not match:
        raise ValueError("Invalid DMS format")

    degrees = int(match.group(1))
    minutes = int(match.group(2))
    seconds = float(match.group(3))
    direction = match.group(4)

    # Convert to decimal degrees
    decimal_degrees = degrees + minutes / 60 + seconds / 3600

    # Adjust sign for South and West
    if direction in 'SW':
        decimal_degrees *= -1

    return decimal_degrees

def addr_to_coords(addr: str) -> (float, float):
    API_KEY = '6cda2bcd35394349886014d2a5c6f497'
    geocoder = auto.opencage.geocoder.OpenCageGeocode(API_KEY)

    data = geocoder.geocode(addr)[0]
    lat = data['annotations']['DMS']['lat']
    lng = data['annotations']['DMS']['lng']
    lat = dms_to_decimal(lat)
    lng = dms_to_decimal(lng)
    return (lat, lng)

def scope():
    print(addr_to_coords('8000 Middlebrook Pike Knoxville TN'))

/scope

(35.943011, -84.05265299999999)


## Lat/lng from point with radius & direction

In [22]:
def get_point_at_distance(lat, lon, distance_miles, bearing_degrees):
    """
    Calculate a new coordinate point given:
    - Starting point (latitude, longitude)
    - Distance in miles
    - Bearing in degrees (0° is North, 90° is East, etc.)
    
    Returns: (new_latitude, new_longitude)
    """
    # Convert to radians
    lat_rad = auto.math.radians(lat)
    lon_rad = auto.math.radians(lon)
    bearing_rad = auto.math.radians(bearing_degrees)
    
    # Earth's radius in miles
    R = 3959
    
    # Calculate angular distance
    d = distance_miles / R
    
    # Calculate new latitude
    new_lat_rad = auto.math.asin(
        auto.math.sin(lat_rad) * auto.math.cos(d) +
        auto.math.cos(lat_rad) * auto.math.sin(d) * auto.math.cos(bearing_rad)
    )
    
    # Calculate new longitude
    new_lon_rad = lon_rad + auto.math.atan2(
        auto.math.sin(bearing_rad) * auto.math.sin(d) * auto.math.cos(lat_rad),
        auto.math.cos(d) - auto.math.sin(lat_rad) * auto.math.sin(new_lat_rad)
    )
    
    # Convert back to degrees
    new_lat = auto.math.degrees(new_lat_rad)
    new_lon = auto.math.degrees(new_lon_rad)
    
    return (new_lat, new_lon)

def scope():
    lat, lng = addr_to_coords('8000 Middlebrook Pike Knoxville TN')
    newpoint_north = get_point_at_distance(lat, lng, 1.0, 0.0)
    print((lat, lng))
    print(newpoint_north)

/scope

(35.943011, -84.05265299999999)
(35.957483285807804, -84.05265299999999)


# Background

## Config

In [31]:
config.address: str = "8000 Middlebrook Pike Knoxville TN"
config.radius: float = 1.0

## Satellite

In [26]:
try:
    _Satellite__12b18eaa
except NameError:
    _Satellite__12b18eaa = {}

#@title Satellite
class Satellite:
    was_cached = None

    def __new__(Satellite, *, row: int, col: int, zoom: int) -> auto.PIL.Image:
        return Satellite.from_disk(row=row, col=col, zoom=zoom)

    @staticmethod
    def Tile(url: str, /) -> tuple[int, int, int]:
        path = auto.pathlib.PurePosixPath(url)
        col = int(path.name)
        path = path.parent
        row = int(path.name)
        path = path.parent
        zoom = int(path.name)

        return zoom, row, col

    @classmethod
    def load(Satellite, zpath: auto.os.PathLike, /):
        zpath = auto.pathlib.Path(zpath)
        with auto.zipfile.ZipFile(zpath, 'r') as arc:
            for name in arc.namelist():
                path = auto.zipfile.Path(arc, name)
                if path.is_dir():
                    continue

                zoom, row, col = Satellite.Tile(str(path.parent))
                with path.open('rb') as f:
                    image = auto.PIL.Image.open(f)
                    image.load()

                __12b18eaa[str(path.parent)] = image

    @classmethod
    def save(Satellite, zpath: auto.os.PathLike, /):
        zpath = auto.pathlib.Path(zpath)
        if zpath.exists():
            ztemp = zpath.parent / f'{zpath.name}.tmp'
            if ztemp.exists():
                ztemp.unlink()
            
            zpath.rename(ztemp)

        with auto.zipfile.ZipFile(zpath, 'w') as arc:
            root = auto.zipfile.Path(arc)

            for url, image in __12b18eaa.items():
                zoom, row, col = Satellite.Tile(url)

                with (path := root / f'{zoom}' / f'{row}' / f'{col}' / f'256.png').open('wb') as f:
                    image.save(f, 'PNG')

        print(f'Wrote {zpath.stat().st_size:,d} bytes to {zpath}')

    @classmethod
    def from_disk(Satellite, *, row: int, col: int, zoom: int) -> auto.PIL.Image:
        root = auto.zipfile.Path('data/TEarth/__satellite.zip')
        # root = auto.zipfile.Path('/mnt/seenas2/data/2023_ORNL_Building_Energy_Models/gen/satellite.zip')
        path = root / f'{zoom}' / f'{row}' / f'{col}' / f'256.png'
        if not path.exists():
            Satellite.was_cached = False
            image = Satellite.from_http(row=row, col=col, zoom=zoom)
            with path.open('wb') as f:
                image.save(f, 'PNG')

        else:
            Satellite.was_cached = True
            with path.open('rb') as f:
                image = auto.PIL.Image.open(f)
                image.load()

        return image

    @classmethod
    def from_http(Satellite, *, row: int, col: int, zoom: int) -> auto.PIL.Image:
        url = config.app.satellite.format(z=zoom, x=col, y=row)
        if url not in __12b18eaa:
            Satellite.was_cached = False

            with auto.requests.request(
                'GET',
                url,
                stream=True,
            ) as response:
                response.raise_for_status()

                image = auto.PIL.Image.open(auto.io.BytesIO(response.content))
                image.load()

            /auto.time.sleep 1
            __12b18eaa[url] = image

        else:
            Satellite.was_cached = True
            image = __12b18eaa[url]

        return image


def scope(args: str, /):
    parser = auto.argparse.ArgumentParser()
    args = parser.parse_args(auto.shlex.split(args))

    _Satellite__12b18eaa.clear()

    # Satellite.load(
    #     'data/__ac3095cf.zip',
    # )

    # Satellite.load(
    #     'data/satellite.zip',
    # )

    Satellite.save(
        'data/TEarth/__satellite.zip',
    )

;scope


Wrote 22 bytes to data/TEarth/__satellite.zip


In [42]:
def lon_to_x(lon, zoom):
    """Convert longitude to tile X coordinate"""
    return ((lon + 180) / 360) * auto.math.pow(2, zoom)

def lat_to_y(lat, zoom):
    """Convert latitude to tile Y coordinate"""
    lat_rad = auto.math.radians(lat)
    return (1 - auto.math.log(auto.math.tan(lat_rad) + 1 / auto.math.cos(lat_rad)) / auto.math.pi) / 2 * auto.math.pow(2, zoom)

def get_optimal_zoom_level(min_lat, max_lat, min_lon, max_lon, max_tiles=4):
    """
    Calculate the optimal zoom level for a bounding box.
    max_tiles parameter limits the number of tiles in either dimension.
    """
    for zoom in range(20, -1, -1):  # Start from max zoom and work down
        # Calculate tile coordinates for bounds
        min_x = auto.math.floor(lon_to_x(min_lon, zoom))
        max_x = auto.math.ceil(lon_to_x(max_lon, zoom))
        min_y = auto.math.floor(lat_to_y(max_lat, zoom))  # Note: y coordinates are inverted
        max_y = auto.math.ceil(lat_to_y(min_lat, zoom))
        
        # Calculate number of tiles needed
        tiles_x = max_x - min_x
        tiles_y = max_y - min_y
        
        # If we're within our tile limit, use this zoom level
        if tiles_x <= max_tiles and tiles_y <= max_tiles:
            return zoom
    
    return 0  # Minimum zoom if no other level is suitable

def get_tiles_for_bounds(min_lat, max_lat, min_lon, max_lon, zoom=None, max_tiles=4):
    """
    Get all tile coordinates (x, y, z) within a bounding box.
    If zoom is not specified, calculates optimal zoom level.
    
    Returns:
        - zoom level used
        - list of (x, y, z) tile coordinates
    """
    # If zoom not specified, calculate optimal zoom
    if zoom is None:
        zoom = get_optimal_zoom_level(min_lat, max_lat, min_lon, max_lon, max_tiles)
    
    # Calculate tile coordinates for bounds
    min_x = auto.math.floor(lon_to_x(min_lon, zoom))
    max_x = auto.math.ceil(lon_to_x(max_lon, zoom))
    min_y = auto.math.floor(lat_to_y(max_lat, zoom))  # Note: y coordinates are inverted
    max_y = auto.math.ceil(lat_to_y(min_lat, zoom))
    
    # Generate list of all tile coordinates
    tiles = []
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            tiles.append((x, y, zoom))
    
    return zoom, tiles

def get_tile_bounds(x, y, zoom):
    """
    Get the bounds of a specific tile (min_lat, max_lat, min_lon, max_lon)
    """
    n = auto.math.pow(2, zoom)
    lon1 = x / n * 360 - 180
    lon2 = (x + 1) / n * 360 - 180
    
    # Convert y to latitude
    lat1 = auto.math.degrees(auto.math.atan(auto.math.sinh(auto.math.pi * (1 - 2 * y / n))))
    lat2 = auto.math.degrees(auto.math.atan(auto.math.sinh(auto.math.pi * (1 - 2 * (y + 1) / n))))
    
    return (min(lat1, lat2), max(lat1, lat2), min(lon1, lon2), max(lon1, lon2))

def coordinate_bounds(pointlist):
    min_lat = 500.0
    min_lng = 500.0
    max_lat = -500.0
    max_lng = -500.0

    for p in pointlist:
        if p[0] < min_lat:
            min_lat = p[0]
        elif p[0] > max_lat:
            max_lat = p[0]
        
        if p[1] < min_lng:
            min_lng = p[1]
        elif p[1] > max_lng:
            max_lng = p[1]

    return ({
        'latmax': max_lat,
        'lngmax': max_lng,
        'latmin': min_lat,
        'lngmin': min_lng,
    })


def scope():
    lat, lng = addr_to_coords(config.address)
    print(lat,lng)

    north = get_point_at_distance(lat, lng, config.radius, 0.0)
    east = get_point_at_distance(lat, lng, config.radius, 90.0)
    south = get_point_at_distance(lat, lng, config.radius, 180.0)
    west = get_point_at_distance(lat, lng, config.radius, 270.0)

    print(f'Center: {lat}, {lng}')
    print(f'north: {north}')
    print(f'east: {east}')
    print(f'south: {south}')
    print(f'west: {west}')

    bounds = coordinate_bounds([north, east, south, west])

    zoom, tiles = get_tiles_for_bounds(
        min_lat=bounds['latmin'],
        min_lon=bounds['lngmin'],
        max_lat=bounds['latmax'],
        max_lon=bounds['lngmax'],
    )
    print(f"Optimal zoom level: {zoom}")
    print(f"Number of tiles needed: {len(tiles)}")
    print("\nTile coordinates (x, y, z):")
    for tile in tiles:
        print(f"x: {tile[0]}, y: {tile[1]}, z: {tile[2]}")
        
        # Get bounds for this specific tile
        tile_bounds = get_tile_bounds(tile[0], tile[1], tile[2])
        print(f"Tile bounds: {tile_bounds}\n")


/scope

35.943011 -84.05265299999999
Center: 35.943011, -84.05265299999999
north: (35.957483285807804, -84.05265299999999)
east: (35.94300967482274, -84.03477718031101)
south: (35.92853871419221, -84.05265299999999)
west: (35.94300967482274, -84.07052881968897)
Optimal zoom level: 15
Number of tiles needed: 20

Tile coordinates (x, y, z):
x: 8731, y: 12873, z: 15
Tile bounds: (35.94243575255425, 35.95132986152265, -84.078369140625, -84.0673828125)

x: 8731, y: 12874, z: 15
Tile bounds: (35.93354064249313, 35.94243575255425, -84.078369140625, -84.0673828125)

x: 8731, y: 12875, z: 15
Tile bounds: (35.92464453144098, 35.93354064249313, -84.078369140625, -84.0673828125)

x: 8731, y: 12876, z: 15
Tile bounds: (35.915747419499695, 35.92464453144098, -84.078369140625, -84.0673828125)

x: 8732, y: 12873, z: 15
Tile bounds: (35.94243575255425, 35.95132986152265, -84.0673828125, -84.056396484375)

x: 8732, y: 12874, z: 15
Tile bounds: (35.93354064249313, 35.94243575255425, -84.0673828125, -84.056396484